In [ ]:
import pandas as pd
import json

def merge(filename:str,station_name:pd.DataFrame) -> pd.DataFrame:
    df1 = pd.read_csv(filename)
    df2 = df1.rename(columns={'trnOpDate':'日期',
                    'staCode':'車站代碼',
                    'gateInComingCnt':'進站人數',
                    'gateOutGoingCnt':'出站人數'})
    df2['日期'] = pd.to_datetime(df2['日期'].astype(str))
    result_df = pd.merge(df2,station_name,left_on='車站代碼',right_on='編碼',how='left')
    return result_df

with open('每日各站進出站人數/車站基本資料集.json',encoding='utf-8') as file:
    json_data:list[dict] = json.load(file)

stations_name = pd.DataFrame(json_data,columns=['stationCode','stationName','stationAddrTw'])
stations_name1 = stations_name.rename(columns={'stationCode':'編碼',
                      'stationName':'車站名稱',
                      'stationAddrTw':'地址'})
stations_name1['編碼'] = stations_name1['編碼'].astype(int)

merge(filename='每日各站進出站人數/每日各站進出站人數2021.csv',station_name=stations_name1)


In [12]:
import os
current_path = os.path.abspath(__name__) #目前執行檔案的絕對路徑
main_path = os.path.dirname(current_path) #取得目前執行的目錄的絕對路徑

'每日各站進出站人數'
dataFolder_path = os.path.join(main_path,'每日各站進出站人數') #取得放資料的目錄
os.listdir(dataFolder_path) #取得資料目錄的檔案名稱


['每日各站進出站人數2020.csv',
 '每日各站進出站人數2021.csv',
 '每日各站進出站人數20190423-20191231.csv',
 '車站基本資料集.json',
 '每日各站進出站人數2023.csv',
 'schema.csv',
 'manifest.csv',
 '每日各站進出站人數2022.csv']